In [70]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [71]:
files = glob.glob('16_8_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

scaler = StandardScaler()


test_df = pd.read_csv('Iris.csv').drop(columns = ['Id'])
test_df['Species'] = test_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

test_y = test_df['Species']
test_x = test_df.drop(columns = ['Species'])
test_x = pd.DataFrame(scaler.fit_transform(test_x), columns = test_x.columns)

order= ['fc1', 'fc2', 'fc3']

In [72]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):

    weights_dict_ = copy.deepcopy(_weights_dict)

    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]

    return weights_dict_

def choose_neurons_matmul(chosen_neurons, _weights_dict):
    weights_dict_ = copy.deepcopy(_weights_dict)

    weights_dict_['weight']['fc1'] = chosen_neurons[0] * weights_dict_['weight']['fc1']

    weights_dict_['weight']['fc2'] = chosen_neurons[1] * weights_dict_['weight']['fc2']
    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'] * chosen_neurons[0].T

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'] * chosen_neurons[1].T

    weights_dict_['bias']['fc1'] = chosen_neurons[0].flatten() * weights_dict_['bias']['fc1'].flatten()
    weights_dict_['bias']['fc2'] = chosen_neurons[1].flatten() * weights_dict_['bias']['fc2'].flatten()

    return weights_dict_


def predict_numeric(img_df_row, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], img_df_row.T) + weights_dict_['bias']['fc1']

    x = np.dot(weights_dict_['weight']['fc2'], x) + weights_dict_['bias']['fc2']

    x = np.dot(weights_dict_['weight']['fc3'], x) + weights_dict_['bias']['fc3']

    return x

def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (16, 4) 	|	 (16,)
Layer 2  	|	 (8, 16) 	|	 (8,)
Layer 3  	|	 (3, 8) 	|	 (3,)


In [73]:
### Validation of results

my_test_weights = choose_neurons_numeric([list(range(16)), list(range(8))], weights_dict)

preds = np.array([predict_numeric(i, my_test_weights) for i in test_x.values])

counter = 0
for i in range(len(preds)):
    if preds[i].argmax() == test_y.iloc[i]:
        counter += 1

print(counter/150)

def cross_entropy_loss(label, y_hat):

    y = np.zeros_like(y_hat)
    y[np.arange(label.shape[0]), label] = 1

    exp_preds = np.exp(y_hat)

    softmaxed_preds = exp_preds / exp_preds.sum(axis=1, keepdims=True)

    return -np.sum(y * np.log(softmaxed_preds)) / len(test_y)


def cross_entropy_loss_cplex(label, y_hat, model):

    y = np.zeros_like(y_hat)
    y[np.arange(label.shape[0]), label] = 1

    exp_preds = np.e**(y_hat)

    softmaxed_preds = exp_preds / exp_preds.sum(axis=1, keepdims=True)

    to_sum = []

    for u, i in zip(y, softmaxed_preds):
      for o, j in zip(u, i):
        to_sum.append(o * m.log(j))


    return -np.sum(to_sum) / len(test_y)


print(cross_entropy_loss(test_y, preds))

0.9733333333333334
0.06532579124930898


### DoCplex

In [74]:
# %%capture
# !pip install docplex cplex

In [101]:

from docplex.cp.model import CpoModel as Model
def choose_and_move(_weights_dict_, log_output = None, neurons_to_remove = 1):
    weights_dict_ = copy.deepcopy(_weights_dict_)
    m = Model()

    neurons_layer1 = len(weights_dict_['bias']['fc1'])
    neurons_layer2 = len(weights_dict_['bias']['fc2'])

    X_fc1 = np.array(m.binary_var_list(neurons_layer1, name='Layer 1 Neurons')).reshape((-1, 1))
    X_fc2 = np.array(m.binary_var_list(neurons_layer2, name='Layer 2 Neurons')).reshape((-1, 1))

    neurons_used = np.sum(X_fc1) + np.sum(X_fc2)

    m.add_constraint(neurons_used == (neurons_layer1 + neurons_layer2 - neurons_to_remove))

    new_weights_dict_ = choose_neurons_matmul([X_fc1, X_fc2], weights_dict_)

    preds = np.array([predict_numeric(i, new_weights_dict_) for i in test_x.values])

    loss = cross_entropy_loss_cplex(test_y, preds, m)

    m.minimize(loss)

    sol = m.solve(log_output=log_output)
    
    layer1 = np.ones(neurons_layer1)
    layer2 = np.ones(neurons_layer2)

    for i in sol.get_all_var_solutions():
        _, layer_number, neuron_number = i.get_name().split(' ')

        neuron_number_int = int(neuron_number.split('_')[-1])

        if layer_number == '1':
            layer1[neuron_number_int] = (i.get_value())

        if layer_number == '2':
            layer2[neuron_number_int] = (i.get_value())

    print('==== Finished Iteration ====')
    print('Loss:', sol.get_objective_values()[0])
    print('Neurons Used:')
    print(np.where(layer1 == 1)[0])
    print(np.where(layer2 == 1)[0])

    return layer1, layer2

In [102]:
layer1, layer2 = None, None
weights_dict_copy_iterative = copy.deepcopy(weights_dict)
for i in range(4):
    layer1, layer2 = choose_and_move(weights_dict_copy_iterative)
    weights_dict_copy_iterative = choose_neurons_numeric([np.where(layer1 == 1)[0], np.where(layer2 == 1)[0]], weights_dict_copy_iterative)

==== Finished Iteration ====
Loss: 0.06547558704562792
Neurons Used:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15]
[0 1 2 3 4 5 6 7]
==== Finished Iteration ====
Loss: 0.0683419462607332
Neurons Used:
[ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
[0 1 2 3 4 5 6 7]
==== Finished Iteration ====
Loss: 0.07218602119035404
Neurons Used:
[ 0  1  2  4  5  6  7  8  9 10 11 12 13]
[0 1 2 3 4 5 6 7]
==== Finished Iteration ====
Loss: 0.07288669835025553
Neurons Used:
[ 0  1  2  3  5  6  7  8  9 10 11 12]
[0 1 2 3 4 5 6 7]


In [103]:

choose_and_move(weights_dict, log_output = None, neurons_to_remove = 3)


==== Finished Iteration ====
Loss: 0.0687947261965805
Neurons Used:
[ 0  1  2  4  6  7  8  9 10 11 12 13 15]
[0 1 2 3 4 5 6 7]


(array([1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))